# Generate KM-adjusted delay distributions

Data downloaded from public CDC linelist on September 9, 2021.

https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data/vbim-akqf


In [ ]:
# standard
import pickle
from datetime import timedelta

# third party
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from pandas import read_csv, date_range, Series
from tqdm.notebook import tqdm

# first party
from config import Config

## Read in linelist data.

In [ ]:
surveil_df = read_csv("../data/COVID-19_Case_Surveillance_Public_Use_Data_20210909.csv",
                      usecols=["cdc_report_dt", "onset_dt"],
                      parse_dates=["cdc_report_dt", "onset_dt"])
surveil_df.onset_dt = surveil_df.onset_dt.dt.date
surveil_df.cdc_report_dt = surveil_df.cdc_report_dt.dt.date

# Remove missing onset rows, and data prior to our assumed first reliable day of data.
linelist = surveil_df[~surveil_df.onset_dt.isna()]
linelist = linelist[linelist.cdc_report_dt.ge(Config.first_data_date)]

## Calculate reporting delays.

In [ ]:
linelist['report_delay'] = (linelist.cdc_report_dt - linelist.onset_dt).dt.days
linelist = linelist[linelist.report_delay.gt(0) & linelist.report_delay.le(Config.max_delay_days)]

In [ ]:
storage_dir = '../data/km_delay_distributions'

In [ ]:
def empirical_pmf(delay_df, 
                  max_delay=Config.max_delay_days,
                  distribution_support=Config.distribution_support):
    delays = delay_df[delay_df.report_delay.le(max_delay)]
    emp_dist = delays.groupby('report_delay').onset_dt.count()
    emp_dist /= emp_dist.sum()
    emp_dist = emp_dist.reindex(distribution_support, fill_value=0)
    return emp_dist

In [ ]:
def construct_training_pmfs(linelist, as_of, Config=Config):
    d = Config.support_size
    window_size = 2*d
    support = Config.distribution_support
        
    last_truncated = as_of - timedelta(Config.max_delay_days)
    truncated_dates = [d.date() for d in date_range(last_truncated, as_of)]
    fair_df = linelist[linelist.cdc_report_dt.lt(as_of)]
    
    pmfs = {}
    for working_onset_date in tqdm(truncated_dates):
        t = as_of
        i = (t - working_onset_date).days
        running_kernel = Series([0]).reindex(support, fill_value=0)
        
        min_date = t - timedelta(i) - timedelta(window_size) + timedelta(1)
        max_date = t - timedelta(d)
        
        trimmed_df = fair_df[fair_df.onset_dt.ge(min_date)]
        D_tilde = trimmed_df[trimmed_df.onset_dt.le(max_date)]
        update = empirical_pmf(D_tilde, d)
        if i == d:
            running_kernel = update
        else:
            running_kernel.loc[d] = update.loc[d]
            survival = update.loc[d]
            for j in range(d-1, i-1, -1):
                max_date = t - timedelta(j)
                D_tilde = trimmed_df[trimmed_df.onset_dt.le(max_date)]
                update = empirical_pmf(D_tilde, j) * (1 - survival)
                if j > i:
                    running_kernel.loc[j] = update.loc[j]
                    survival += update.loc[j]
                else:
                    running_kernel.loc[:j] = update.loc[:j]

        assert np.isclose(running_kernel.sum(), 1), working_onset_date
        
        # Fit gamma distribution
        mu = (running_kernel*support).sum()
        var = (running_kernel*(support**2)).sum() - mu**2
        gam = stats.gamma(mu**2 / var, loc=0, scale=(var / mu))
        delay_dist = np.array([gam.cdf(i+1) - gam.cdf(i) for i in support])
        delay_dist /= delay_dist.sum()
        pmfs[working_onset_date] = np.r_[0, delay_dist] # Add pr 0 at lag=0

    return pmfs

In [ ]:
for as_of in Config.as_of_range:
    pmfs = construct_training_pmfs(linelist, as_of)
    pickle.dump(pmfs, open(f'{storage_dir}/delay_distribution_as_of_{as_of}.p', 'wb'),
               protocol=pickle.HIGHEST_PROTOCOL)

### Add extra past

- For each working date `s` older than `d` days, we have fully observed all the possible reporting dates (no need to truncate by report date). Hence, we can simply take the rows where the symptom onset date falls in `[s - 2*d + 1, s]`. We first construct all of these pmfs, and then fill in the extra past for the training kernels.

In [ ]:
fully_observed_pmfs = pickle.load(open('../data/naive_delay_distributions/uncensored_delay_distribution.p', 'rb'))

In [ ]:
# Fill in delay distribution pickles
first_data_date = Config.first_data_date
d = Config.max_delay_days
for run_date in tqdm(Config.as_of_range):
    try:
        pmfs = pickle.load(open(f'{storage_dir}/delay_distribution_as_of_{run_date}.p', 'rb'))
    except Exception as e:
        print(run_date, "missing")
        continue

    first_uncensored_date = run_date - timedelta(d) - timedelta(1)
    if first_uncensored_date <= first_data_date:
        print(run_date)
        continue 
    
    uncensored_range = [d.date() for d in date_range(first_data_date, first_uncensored_date)]
    existing_working_dates = sorted(pmfs.keys())
    assert run_date == existing_working_dates[-1]
    
    for working_onset_date in uncensored_range:
        pmfs[working_onset_date] = fully_observed_pmfs[working_onset_date]
        
    assert len(pmfs) == (run_date - first_data_date).days + 1
    pickle.dump(pmfs, 
                open(f'{storage_dir}/delay_distribution_as_of_{run_date}.p', 'wb'), 
                protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Plot example of error
naive = pickle.load(open(f'../data/naive_delay_distributions/delay_distribution_as_of_{as_of}.p', 'rb'))
truth = pickle.load(open(f'../data/naive_delay_distributions/uncensored_delay_distribution.p', 'rb'))

err_adjusted = []
err_naive = []
for i in range(50):
    a = as_of - timedelta(i)
    err_adjusted.append(np.sum(np.abs(pmfs[a] - truth[a])))
    err_naive.append(np.sum(np.abs(naive[a] - truth[a])))
plt.plot(err_adjusted, color='tab:blue', label='km')
plt.plot(err_naive, color='tab:orange', label='naive')
plt.legend()